In [5]:
import json
import pandas as pd
import os

os.listdir('open-data/data/')

['lineups', 'matches', 'competitions.json', 'events']

In [11]:
import os
event_data_path = "open-data/data/events/"
event_files = os.listdir(event_data_path)

count = 0
dfs = [] # an empty list to store the data frames
for event_file in event_files:
    event_id = event_file.split(".")[0]
    count += 1
    event_file = open(event_data_path + event_file,encoding='UTF-8')
    data = json.load(event_file)
    df = pd.json_normalize(data)
#     print(df.shape)
#     foul_outcome = df.loc[df['type.name'] == 'Foul Committed']
    df["event_id"] = event_id
#     print(df.shape, foul_outcome.shape)
    dfs.append(df) # append the data frame to the list

all_subs_data = pd.concat(dfs, ignore_index=True)

In [18]:
for col in all_subs_data.columns:
    print(col)

id
index
period
timestamp
minute
second
possession
duration
type.id
type.name
possession_team.id
possession_team.name
play_pattern.id
play_pattern.name
team.id
team.name
tactics.formation
tactics.lineup
related_events
location
player.id
player.name
position.id
position.name
pass.recipient.id
pass.recipient.name
pass.length
pass.angle
pass.height.id
pass.height.name
pass.end_location
pass.type.id
pass.type.name
pass.body_part.id
pass.body_part.name
under_pressure
carry.end_location
out
counterpress
pass.outcome.id
pass.outcome.name
ball_receipt.outcome.id
ball_receipt.outcome.name
clearance.right_foot
clearance.body_part.id
clearance.body_part.name
duel.type.id
duel.type.name
pass.aerial_won
foul_committed.type.id
foul_committed.type.name
duel.outcome.id
duel.outcome.name
dribble.outcome.id
dribble.outcome.name
dribble.overrun
goalkeeper.outcome.id
goalkeeper.outcome.name
goalkeeper.type.id
goalkeeper.type.name
clearance.left_foot
pass.assisted_shot_id
pass.shot_assist
shot.statsbomb_xg

In [29]:
all_subs_data.loc[pd.isnull(all_subs_data.location)].groupby(["type.name"]).count().columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'possession',
       'duration', 'type.id', 'possession_team.id',
       ...
       'half_start.late_video_start', 'goalkeeper.success_in_play',
       'goalkeeper.success_out', 'shot.follows_dribble', 'goalkeeper.lost_out',
       'goalkeeper.penalty_saved_to_post', 'goalkeeper.saved_to_post',
       'half_end.early_video_end', 'player_off.permanent', 'shot.kick_off'],
      dtype='object', length=149)

In [19]:
# all_subs_data.to_csv('all.csv', index = False, header=True)

In [42]:
import os
event_data_path = "data/events/"
event_files = os.listdir(event_data_path)
count = 0
dfs = [] # an empty list to store the data frames
for event_file in event_files:
    event_id = event_file.split(".")[0]
    count += 1
    event_file = open(event_data_path + event_file,encoding='UTF-8')
    data = json.load(event_file)
    df = pd.json_normalize(data)
    substitutions_outcome = df.loc[df['substitution.outcome.name'].notnull()]
    substitutions_outcome["event_id"] = event_id
    dfs.append(substitutions_outcome) # append the data frame to the list

all_substitution_data = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.



In [43]:
all_substitution_data.shape

(5364, 150)

In [44]:
injury_subs_data = all_substitution_data.loc[all_substitution_data['substitution.outcome.name'] == "Injury"]

In [45]:
# Get potential injury substitutions
import warnings
warnings.filterwarnings("ignore")
event_data_path = "data/events/"
event_files = os.listdir(event_data_path)
count = 0
injury_df = [] # an empty list to store the data frames
for event_file in event_files:
    event_id = event_file.split(".")[0]
    count += 1
    # data = pd.read_json(file, lines=True) # read data frame from json file
    event_file = open(event_data_path + event_file,encoding='UTF-8')
    data = json.load(event_file)
    # data = pd.read_json(event_data_path + event_file, lines=True)
    df = pd.json_normalize(data)

    tactical_substitutions = df.loc[df['substitution.outcome.name'] == 'Tactical']
    injury_stoppage = df.loc[df["type.name"] == "Injury Stoppage"]
    injury_stoppage_mergedf = injury_stoppage[["player.name", "minute"]]
    injury_stoppage_mergedf.rename(columns={"minute": "minute_y"}, inplace=True)
    injury_stoppage_mergedf["injury_limit"] = injury_stoppage_mergedf["minute_y"] + 10
    injury_stoppage_mergedf
    new_df = pd.merge(tactical_substitutions, injury_stoppage_mergedf,  how='inner', left_on=['player.name'], right_on = ['player.name'])
    new_df = new_df[(new_df["minute"] <= new_df["injury_limit"]) & (new_df["minute"] >= new_df["minute_y"])]
    new_df["event_id"] = event_id
    injury_df.append(new_df)

possible_injury_subs_data = pd.concat(injury_df, ignore_index=True) # concatenate all the data frames in the list.

In [46]:
possible_injury_subs_data_merge = possible_injury_subs_data.drop(["minute_y","injury_limit"], axis = 1)
possible_injury_subs_data_merge.shape

(157, 150)

In [48]:
all_injuries = pd.concat([possible_injury_subs_data_merge, injury_subs_data], ignore_index=True, sort=False)
all_injuries

,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,...,goalkeeper.shot_saved_to_post,goalkeeper.lost_out,player_off.permanent,goalkeeper.lost_in_play,goalkeeper.success_out,goalkeeper.success_in_play,goalkeeper.saved_to_post,half_end.early_video_end,shot.kick_off,goalkeeper.penalty_saved_to_post
0,e9aedc0e-afea-42d3-bcfc-5fd744e9d9ed,2685,2,00:17:55.394,62,55,109,0.00,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a6f430d6-a5e5-4317-b55a-e1cadd0afc7f,3702,2,00:41:53.704,86,53,164,0.00,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,222c0a40-bc5f-4649-baad-86764a371613,1982,2,00:00:00.000,45,0,84,0.00,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f64673c4-2315-4a8c-b128-6d990076c97e,1937,2,00:00:00.000,45,0,118,0.00,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29f59450-bde4-4202-9f17-165623ad3c82,3279,2,00:31:37.192,76,37,191,0.00,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,b78a7b2b-4ddf-4bba-b2ef-f00f137aaf96,2005,1,00:38:38.333,38,38,59,NaN,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
449,9acde55c-2529-4280-bd37-dcf356ada57b,3467,2,00:26:33.879,71,33,147,0.00,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,a850a657-efef-4db1-b94f-66e7fbdd051c,2305,2,00:09:42.206,54,42,113,0.00,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451,7517d55d-9515-497a-ac5e-742e3b3e1bad,1746,1,00:42:40.556,42,40,81,0.00,19,Substitution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
all_injuries.sort_values(by=['period','timestamp'], inplace=True)
all_injuries = all_injuries.reset_index(drop=True)
all_injuries.to_csv('all_injuries_wevent_id.csv', index = False, header=True)

In [51]:
# set a new column related substitution
import numpy as np
all_subs_data['related_substitution'] = np.nan

In [52]:
all_subs_data['related_substitution_timestamp'] = np.nan

In [59]:
# link the injury to fouls  
from dateutil.parser import parse
time_gap = []
for _, row in all_injuries.iterrows():
    event_id = row['event_id']
    period = row['period']
    time = parse(row['timestamp'])
    foul_slice = all_subs_data.loc[(all_subs_data['event_id'] == event_id) & (all_subs_data['period'] == period) ]
    
    for _, foul in foul_slice.iterrows():
        foul_time = parse(foul['timestamp'])
        if time > foul_time:
            id = foul['id']
            
            all_subs_data.related_substitution[all_subs_data['id'] == id] = row['id']
            all_subs_data.related_substitution_timestamp[all_subs_data['id'] == id] = row['timestamp']
            time_gap.append((time-foul_time).total_seconds())
            break
            

    

In [60]:
# Try to calculate the time gap between foul and substitution
sum(time_gap)/len(time_gap)

1459.3661602870804